In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
raw_data = pd.concat([pd.read_csv(file) for file in ["./raw/FR_2021_hourly.csv", "./raw/FR_2022_hourly.csv"]])

In [4]:
data = raw_data[['Datetime (UTC)', 'Carbon Intensity gCO₂eq/kWh (direct)']].copy()
data['Datetime'] = pd.to_datetime(data['Datetime (UTC)']) - pd.DateOffset(years=14)
data.set_index('Datetime', inplace=True)
data.drop(columns=['Datetime (UTC)'], inplace=True)
data["Carbon Intensity"] = data["Carbon Intensity gCO₂eq/kWh (direct)"]
data.drop(columns=['Carbon Intensity gCO₂eq/kWh (direct)'], inplace=True)

In [5]:
data_minutely = data.resample('min').interpolate(method='linear')
data_minutely["Carbon Intensity"] /= 60
data_hourly = data

In [6]:
for data, resolution in zip([data_minutely, data_hourly], ["minutely", "hourly"]):
    print(f"{resolution} Max: ", np.max(data['Carbon Intensity']))
    print(f"{resolution} Min: ", np.min(data['Carbon Intensity']))

    episodes = data.groupby(pd.Grouper(freq='W'))
    required_length = 10000 if resolution == "minutely" else 165
    episodes = pd.concat([group.fillna(0) for name, group in episodes if
                                       len(group.dropna()) >= required_length and group.index[0] >= datetime(2007, 1, 1) and
                                       group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))
    print(f"{resolution} episodes: ", len(episodes))
    demand = pd.HDFStore(f'./{resolution}/household_energy_demand.h5')
    with pd.HDFStore(f'./{resolution}/carbon_intensity.h5') as store:
        # Save each DataFrame with a key
        i = 0
        for name, group in episodes:
            if name.date() == demand[f"eps_{i}"].index[-1].date():
                store[f'eps_{i}'] = group
                i += 1
        print(len(store))
    print(f"{resolution} episodes: ", i)
    file = pd.HDFStore(f'./{resolution}/carbon_intensity.h5')
    print(f"Equal length: {len(file) == len(demand)}")
    file.close()
    demand.close()

minutely Max:  2.3729999999999998
minutely Min:  0.051166666666666666
minutely episodes:  104
101
minutely episodes:  101
Equal length: True
hourly Max:  142.38
hourly Min:  3.07
hourly episodes:  104
102
hourly episodes:  102
Equal length: True
